There is a lack of affordable access to many important medications in the Arab world. Some of these medicationsa are available in neighboring countries at reasonable prices. If these medications can be distributed effectively taking advantage of existing transport between the countries, then these crucial medications can be made more plentiful and affordable. To minimize the cost of medications and take advantage of this opportunity for arbitrage, transport of medications from airports to consumers must be as efficient as possible. We solve this problem as a vehicle routing problem where the objective is to minimize the overall cost of deliveries given a fleet of vehicles with limited capacities. We formulate this as a quantum constrained binary optimization problem which is solved using quantum annealing with D-Wave's superconducting hardware.

In [56]:
from dimod import (
    Binary,
    BinaryQuadraticModel,
    ConstrainedQuadraticModel,
    quicksum,
)

from dwave.system import LeapHybridCQMSampler

import random
import pandas as pd
import numpy as np

# Insert token here to run
#TOKEN = ""

from credentials import TOKEN

from process_output import get_routes_from_sample, get_cost_routes, report_output

First, consider the case where there is a single airport where medication can be delivered and the medication must be delivered at the lowest cost possible by a fleet of delivery vehicles. These vehicles are limited in capacity and how far they can travel in one day. To solve this problem efficiently with VQE, we first formalize it as a constrained optimization problem:

Consider when there are $M$ vehicles and $N$ delivery locations. 

Let $x_{i, j, k}$ be an indicator variable such that $x_{i, j, k} = 1$ if vehicle $i$ visits dropoff location $j$ at stop number $k$ on its route and $x_{i, j, k} = 0$ otherwise.

Number each of the delivery locations $1$ to $N+1$ where the final location is the airport where the vehicles must start and end.

Since each vehicle must start at the airport, the cost for all vehicles to go to the first destination is 
$$\sum\limits_{m=1}^M\sum\limits_{n=1}^N x_{m,n,1}C_{N+1, n}$$
the cost of each vehicle to go from its final destination back to the airport is
$$\sum\limits_{m=1}^M\sum\limits_{n=1}^N x_{m,n,N}C_{n, N+1}$$
and the cost of the vehicle to go between all of its destinations in the middle is
$$\sum\limits_{m=1}^M\sum\limits_{n=1}^{N-1}\sum\limits_{i=1}^{N+1}\sum\limits_{j=1}^{N+1} x_{m,i,n}x_{m,j,n+1}C_{i,j}$$

By combining these terms, we can achieve the objective function for this problem:
$$\sum\limits_{m=1}^M\sum\limits_{n=1}^N x_{m,n,1}C_{N+1, n} + \sum\limits_{m=1}^M\sum\limits_{n=1}^N x_{m,n,N}C_{n, N+1} + \sum\limits_{m=1}^M\sum\limits_{n=1}^{N-1}\sum\limits_{i=1}^{N+1}\sum\limits_{j=1}^{N+1} x_{m,i,n}x_{m,j,n+1}C_{i,j}$$

However, we must also include practical constraints. First, we want the vehicles to drop off all of the medication, which means that each vertex must be visited at least once. Since the input to this method is a fully connected graph that follows the triangle inequality, it follows that each vertex should only be visited only once. Thus we add the constraint

$$\forall j \in [N+1], \sum\limits_{m=1}^M\sum\limits_{k=1}^{N+1} x_{m,j,k} = 1$$

Next, it is only feasible for a vehicle to be in one location at a time. This constraint can be represented as

$$\forall i \in [M], k \in [N+1], \sum\limits_{j=1}^{N+1} x_{i, j, k} == 1$$

Finally, vehicles are constrained in how far they can drive in one trip. If vehicle $i$ can travel at must $d_i$, this constraint can be expressed as

$$\forall i \in [M], \sum\limits_{a=1}^{N+1}\sum\limits_{b=1}^{N+1}\sum\limits_{k=1}^N x_{i,a,k}x_{i,b,k+1}C_{i,j} \leq d_i$$

In [57]:
def build_vrp_cqm(num_locations, distances, num_vehicles, max_distance):
    """
    Input:
        num_locations: number of distinct locations for orders
        distances: an matrix with the distances between respective locations for a fully connected graph
        num_vehicles: the number of vehicles used
        max_distance: the maximum distance that a truck can drive in one day
    Output:
        A constrained quadratic model representing the problem
    """

    M = num_vehicles
    N = num_locations

    cqm = ConstrainedQuadraticModel()

    # Create all the variables: one for each vehicle/location/position combo
    # k is timestep, j is vertex, i is vehicle
    x = {(i, j, k): Binary(str(i) + "_" + str(j) + "_" + str(k)) for k in range(N) for j in range(N+1) for i in range(M)}

    # Define the unconstrained binary optimization problem
    obj = BinaryQuadraticModel(vartype="BINARY")

    # The cost of going from the depot to the first stop
    for m in range(M):
        for n in range(N):
            obj += x[m, n, 0] * distances[N][n]

    # The cost of going from the last stop to the depot
    for m in range(M):
        for n in range(N):
            obj += x[m, n, N-1] * distances[n][N]

    # The cost of going between all stops in the middle
    for m in range(M):
        for n in range(N-1):
            for i in range(N+1):
                for j in range(N+1):
                    obj += x[m, i, n] * x[m, j, n + 1] * distances[i][j]

    cqm.set_objective(obj)

    # Implement constraints:

    # 1. Each location should be served by exactly one vehicle (only checks first N because depot is required start and end location by construction)
    for j in range(N):
        sum = 0
        for m in range(M):
            for k in range(N):
                sum += x[m, j, k]

        cqm.add_constraint(sum == 1,
                           label=f"Vertex {j} is not visited or visited more than once")

    # 2. Each vehicle is in one location
    for i in range(M):
        for k in range(N):
            sum = 0
            for j in range(N + 1):
                sum += x[i, j, k]
            cqm.add_constraint(sum == 1,
                               label=f"Vehicle {i} is at more or less than one position at time {k}")
            
    #3. Each vehicle drives less than the cap
    for m in range(M):
        sum = 0

        for i in range(N+1):
            
            # Add in the distances from deport to first, last to depot
            sum += x[m, i, 0]*distances[N][i]
            sum += x[m, i, N-1]*distances[i][N]

            # Go through the steps in the middle
            for j in range(N+1):
                for k in range(N-1):
                    sum += x[m, i, k]*x[m, j, k+1]*distances[i][j]

        cqm.add_constraint(sum <= max_distance,
                            label=f"Vehicle {m} drives more than the maximum capacity")

            
    # Return the constrained optimization solution
    return cqm

In [58]:
def run_cqm(cqm):
    """Run the provided CQM on the Leap Hybrid CQM Sampler."""
    sampler = LeapHybridCQMSampler(token=TOKEN)

    sampleset = sampler.sample_cqm(cqm)
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    num_feasible = len(feasible_sampleset)
    errors = " "
    if num_feasible == 0:
        print("No feasible solution found.")
        return sampleset

    print("\nFeasible solution found.\n")

    return feasible_sampleset

In [59]:
# Create a sample problem
num_destinations = 5
num_vehicles = 1
max_distance = 300

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        # Select random values that do not violate the triangle inequality
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(5,9)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

# Print the adjacency matrix
for row in cost_matrix:
    print(row)

cqm = build_vrp_cqm(num_destinations, cost_matrix, num_vehicles, max_distance)

[0, 6, 7, 9, 8, 9]
[6, 0, 5, 5, 9, 5]
[7, 5, 0, 9, 6, 9]
[9, 5, 9, 0, 9, 5]
[8, 9, 6, 9, 0, 8]
[9, 5, 9, 5, 8, 0]


In [60]:
feasible_sampleset = run_cqm(cqm)


Feasible solution found.



In [61]:
lowest_energy_sample = feasible_sampleset.lowest().first.sample

routes = get_routes_from_sample(lowest_energy_sample, num_vehicles, num_destinations)

print(f'Best route: {routes}')

print(f'Best cost: {feasible_sampleset.lowest().first.energy}')

#print(get_cost_routes(routes, cost_matrix))

# print(feasible_sampleset)

Best route: [[3, 1, 0, 2, 4]]
Best cost: 37.0


In [62]:
# Test the solver on an instance with more than one vehicle
cqm2 = build_vrp_cqm(num_destinations, cost_matrix, num_vehicles=2, max_distance=30)

In [63]:
feasible_sampleset2 = run_cqm(cqm2)


Feasible solution found.



In [64]:
def parse_string(input_string):
    return list(map(int, input_string.split('_')))

def get_routes_from_sample(sample, num_vehicles, num_steps):
    """Builds a set of routes from the sample returned."""

    routes =  [[-1]*num_steps for _ in range(num_vehicles)]

    # Go through all entries
    for key, val in sample.items():
        vehicle, vertex, step = parse_string(key)
        if val == 1.0:
            routes[vehicle][step] = vertex

    # Clean up trailing and leading values (not optimized)
    for route in routes:
        while route:
            if route[0] == 5:
                route.pop(0)
            else:
                break
        while route:
            if route[-1] == 5:
                route.pop(len(route) - 1)
            else:
                break
    
    return routes

In [67]:
lowest_energy_sample2 = feasible_sampleset2.lowest().first.sample

routes2 = get_routes_from_sample(lowest_energy_sample2, num_vehicles=2, num_steps=num_destinations)

report_output(routes2, cost_matrix)

Best routes (depot omitted at start and end):
	Vehicle 0: [4, 2, 0]
	Vehicle 1: [3, 1]
Best cost: 45


In [68]:
# Lazy sanity check
r = []
for i in range(5):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    r1 = r[:2]
    r2 = r[2:]

    cost = get_cost_routes([r1, r2], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r1, r2]
print(min)
print(best_routes)

45
[[3, 1], [0, 2, 4]]
